In [1]:
base_dir = "/home3/skaasyap/willett"
batch_size=8
device = 'cuda:0'
# sentence_path = '/home3/skaasyap/my_list.pkl'

import numpy as np
import neural_decoder.lm_utils as utils
from neural_decoder.dataset import getDatasetLoaders
# from neural_decoder.load_models import loadModel, loadTransformerModel, run_forward, run_ngram_model, convert_sentence, original_forward
import pickle
import wandb
import os

# with open(sentence_path, 'rb') as f:
#     ground_truth_sentences = pickle.load(f)



/home3/ebrahim2/miniconda3/envs/speech-bci/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
lmDir = base_dir+'/lm/languageModel'
ngramDecoder = utils.build_lm_decoder(
    lmDir,
    acoustic_scale=0.8, #1.2
    nbest=1,
    beam=18
)

I0509 10:46:24.108592 1140766 brain_speech_decoder.h:52] Reading fst /home3/skaasyap/willett/lm/languageModel/TLG.fst
I0509 10:48:51.937197 1140766 brain_speech_decoder.h:81] Reading symbol table /home3/skaasyap/willett/lm/languageModel/words.txt


In [4]:
modelPath = '/home3/skaasyap/willett/outputs/'
modelNames = ['neurips_transformer_time_masked_seed_5']
dataPath_list = ['/home3/skaasyap/willett/data_log_both']
model_types=['t']
ground_truths = []
ground_truths.append(ground_truth_sentences)
# modelNames = ['neurips_transformer_time_masked_seed_0', 'neurips_gru_long_runs_seed_1', 'neurips_gru_long_runs_seed_2', 'neurips_gru_long_runs_seed_3']
# dataPath_list = ['/home3/skaasyap/willett/data', '/home3/skaasyap/willett/data', '/home3/skaasyap/willett/data', '/home3/skaasyap/willett/data']
# model_types = ['r', 'r', 'r', 'r'] #transformer or rnn for each path

NameError: name 'ground_truth_sentences' is not defined

In [6]:
# model_dirs = ["/home3/skaasyap/willett/outputs/", "/home3/skaasyap/willett/outputs/",  "/home3/skaasyap/willett/outputs/", "/home3/skaasyap/willett/outputs/"]
# model_names = ["neurips_transformer_ablation_no_data_log_seed_", "neurips_transformer_ablation_no_AdamW_seed_", "neurips_transformer_time_masked_10_15_seed_", "neurips_transformer_time_masked_15_10_seed_"]
# data_paths = ["/home3/skaasyap/willett/data", "/home3/skaasyap/willett/data_log_both", "/home3/skaasyap/willett/data_log_both", "/home3/skaasyap/willett/data_log_both"]
# model_types = ['t', 't', 't', 't']
# restricted_days_vals = [[], [], [], []]
# num_models_counts = [4, 4, 4, 4]
model_dirs = ["/home3/skaasyap/willett/obi_models/outputs/"]
model_names = ["neurips_gru_data_log_time_masked_lr_schedule_seed_"]
data_paths = ["/home3/skaasyap/willett/data_log_both"]
model_types = ['r']
restricted_days_vals = [[]]
num_models_counts = [9]

In [7]:
model_dir_list = []
model_name_list = []
data_path_list = []
restricted_days_list = []
model_types_list = []
for i in range(len(num_models_counts)):
    for j in range(num_models_counts[i]):
        model_dir_list.append(model_dirs[i])
        model_name_list.append(model_names[i] + str(j))
        data_path_list.append(data_paths[i])
        restricted_days_list.append(restricted_days_vals[i])
        model_types_list.append(model_types[i])
        

# for i in range(0, 4):
#     model_name = "neurips_transformer_time_masked_restricted_days_seed_" + str(i)
#     modelNames.append(model_name)
#     dataPath_list.append("/home3/skaasyap/willett/data_log_both")
#     model_types.append('t')

# for i in range(0, 4):
#     model_name = "neurips_transformer_time_masked_ventral_6v_only_seed_" + str(i)
#     modelNames.append(model_name)
#     dataPath_list.append("/home3/skaasyap/willett/data_log_both")
#     model_types.append('t')

In [5]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from dataset import SpeechDataset

def getDatasetLoadersLocal(
    datasetName,
    batchSize, 
    restricted_days=[]
):
    with open(datasetName, "rb") as handle:
        loadedData = pickle.load(handle)

    def _padding(batch):
        X, y, X_lens, y_lens, days = zip(*batch)
        X_padded = pad_sequence(X, batch_first=True, padding_value=0)
        y_padded = pad_sequence(y, batch_first=True, padding_value=0)

        return (
            X_padded,
            y_padded,
            torch.stack(X_lens),
            torch.stack(y_lens),
            torch.stack(days),
        )
        
  
    train_ds = SpeechDataset(loadedData["train"], transform=None, restricted_days=restricted_days)
    test_ds = SpeechDataset(loadedData["test"], restricted_days=restricted_days)
    comp_data = SpeechDataset(loadedData["competition"], restricted_days = restricted_days)

    train_loader = DataLoader(
        train_ds,
        batch_size=batchSize,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
        collate_fn=_padding,
    )
        
    test_loader = DataLoader(
        test_ds,
        batch_size=batchSize,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        collate_fn=_padding,
    )

    comp_loader = DataLoader(
        comp_data,
        batch_size=batchSize,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        collate_fn=_padding,
    )

    return train_loader, test_loader, comp_loader, loadedData

In [11]:
print(len(loadedData['test']))

24


In [5]:
with open("/home3/skaasyap/willett/data_log_both", "rb") as handle:
    loadedData = pickle.load(handle)
restricted_days = [15, 16, 17, 18, 19, 20]
ground_truth_sentences = []
for i in range(len(loadedData['test'])):
    if i not in restricted_days:
        continue
    cur_data = loadedData['test'][i]
    for i in range(len(cur_data['transcriptions'])):
        ground_truth_sentences.append(cur_data['transcriptions'][i])

In [8]:
print(len(ground_truth_sentences))

240


In [ ]:
model_dir_list = []
model_name_list = []
data_path_list = []
restricted_days_list = []
model_types_list = []

In [6]:
for i in range(len(model_name_list)):
    
    modelName = model_name_list[i]
    modelPath = model_dir_list[i]
    dataPath = data_path_list[i]
    restricted_days = restricted_days_list[i]
    print(f"Runnning for {modelName}")
    
    fullPath = f"{modelPath}{modelName}"
    
    args_file = os.path.join(fullPath, "args")
    with open(args_file, "rb") as f:
        args = pickle.load(f)

    trainLoader, testLoader, loadedData = getDatasetLoaders(
        dataPath,
        batch_size,
        restricted_days=restricted_days,
    )

    wandb.init(project="Language Model", entity="skaasyap-ucla", config=dict(args), name=modelName)

    if(model_types_list[i] == 't'):
        model, args = loadTransformerModel(fullPath)
        model = model.to(device)
        all_logits, trial_lengths, avgDayLoss, cer = run_forward(model, testLoader, device)
        
    else:
        model, args = loadModel(fullPath, device=device)
        model = model.to(device)
        all_logits, trial_lengths, avgDayLoss, cer = original_forward(model, testLoader, device, comp=False)
        
    
    all_logits = [l.cpu().numpy().astype('float32') for l in all_logits]
    trial_lengths = [l.cpu().numpy().astype('int') for l in trial_lengths]
    all_logits = [l for batch in all_logits for l in list(batch)]
    trial_lengths = [l for batch in trial_lengths for l in list(batch)]
    blank_penalty = np.log(2)
    llm_outputs = run_ngram_model(all_logits, trial_lengths, ngramDecoder, blank_penalty)

    print(len(llm_outputs), len(ground_truth_sentences))

    for i in range(len(ground_truth_sentences)):
        ground_truth_sentences[i] = ground_truth_sentences[i].strip()
        llm_outputs[i] = llm_outputs[i].strip()

    for i in range(len(ground_truth_sentences)):
        ground_truth_sentences[i] = convert_sentence(ground_truth_sentences[i])

    cer, wer = utils._cer_and_wer(llm_outputs, ground_truth_sentences, outputType='speech_sil', returnCI=True)

    print(f"cer: {cer}") # cer average, confidence interval 
    print(f"wer: {wer}") #wer average, confidence interval
    
    wandb.log({'cer': cer, 'wer': wer})
    
    out_file = os.path.join(fullPath, "llm_outputs")   # no extension per your spec
    os.makedirs(os.path.dirname(out_file), exist_ok=True)
    with open(out_file, "w", encoding="utf-8") as f:
       f.write("\n".join(llm_outputs))   # one line per LLM output

    out_file_stats = os.path.join(fullPath, "stats")
    os.makedirs(os.path.dirname(out_file_stats), exist_ok=True)
    with open(out_file_stats, "w", encoding="utf-8") as f:
        f.write(str(cer) + '\n')
        f.write(str(wer) + '\n')


Runnning for neurips_gru_data_log_time_masked_lr_schedule_seed_0


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: skaasyap (brain_mae) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home3/skaasyap/miniconda3/envs/willett/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home3/skaasyap/willett/neural_seq_decoder/src/neural_decoder/augmentations.py:170: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return self.conv(input, weight=self.weight, groups=self.groups, padding="same")
/home3/skaasyap/willett/neural_seq_decoder/src/neural_decoder/load_models.py:128: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor

ctc loss: 0.741696, cer: 0.161033
880 880
cer: (0.11804016997784476, 0.1091026106612232, 0.12712668361360016)
wer: (0.1740316421167485, 0.16068189230969823, 0.1879085264841552)
Runnning for neurips_gru_data_log_time_masked_lr_schedule_seed_1


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


...

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


/home3/skaasyap/willett/neural_seq_decoder/src/neural_decoder/load_models.py:128: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),


ctc loss: 0.756078, cer: 0.158805
880 880
cer: (0.12032833327279992, 0.11156078415551725, 0.12951264779429655)
wer: (0.17694126204764501, 0.16404445237743784, 0.19044222100143882)
Runnning for neurips_gru_data_log_time_masked_lr_schedule_seed_8


/home3/skaasyap/willett/neural_seq_decoder/src/neural_decoder/load_models.py:128: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),


ctc loss: 0.726974, cer: 0.159383
880 880
cer: (0.11716848872262375, 0.10799168892752098, 0.12648662794115534)
wer: (0.17203127841425714, 0.15836888875650346, 0.18590816136828406)


In [25]:
trainLoader, testLoader, compLoader, loadedData = getDatasetLoadersLocal(
        dataPath,
        batch_size,
    )

In [19]:
with open('/home3/skaasyap/willett/data', "rb") as handle:
    test = pickle.load(handle)

In [22]:
test_comp = test['competition']
print(len(test_comp))

15


In [27]:
for X, y, X_len, y_len, testDayIdx in compLoader:
    print(testDayIdx)

tensor([0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1])
tensor([2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2])
tensor([3,

In [16]:
print(dataPath)

/home3/skaasyap/willett/data_log_both


In [3]:
fullPath = "/home3/skaasyap/willett/speechBaseline4"
args_file = os.path.join(fullPath, "args")
with open(args_file, "rb") as f:
    args = pickle.load(f)

In [4]:
print(args)

{'outputDir': '/data/willett_data/outputs/speechBaseline4', 'datasetPath': '/data/willett_data/ptDecoder_ctc', 'seqLen': 150, 'maxTimeSeriesLen': 1200, 'batchSize': 64, 'lrStart': 0.02, 'lrEnd': 0.02, 'nUnits': 1024, 'nBatch': 10000, 'nLayers': 5, 'seed': 0, 'nClasses': 40, 'nInputFeatures': 256, 'dropout': 0.4, 'whiteNoiseSD': 0.8, 'constantOffsetSD': 0.2, 'gaussianSmoothWidth': 2.0, 'strideLen': 4, 'kernelLen': 32, 'bidirectional': True, 'l2_decay': 1e-05}


In [12]:
# for i, (modelName, dataPath) in enumerate(zip(modelNames, dataPath_list)):

modelPath = '/home3/skaasyap/willett/obi_models/outputs/'
modelName_base = 'neurips_gru_data_log_time_masked_lr_schedule_seed_0'
dataPath = '/home3/skaasyap/willett/data_log_both'
model_types = []
model_types.append('r')
device = 'cuda:3'
i=0

for i in range(0, 10):
    modelName = modelName_base + str(i)
    print(f"Runnning for {modelName}")

    fullPath = f"{modelPath}{modelName}"

    args_file = os.path.join(fullPath, "args")
    with open(args_file, "rb") as f:
        args = pickle.load(f)
        
    trainLoader, testLoader, compLoader, loadedData = getDatasetLoadersLocal(
        dataPath,
        batch_size,
    )

    wandb.init(project="Language Model", entity="skaasyap-ucla", config=dict(args), name=modelName)

    if(model_types[0] == 't'):
        model, args = loadTransformerModel(fullPath)
        model = model.to(device)
        all_logits, trial_lengths, avgDayLoss, cer = run_forward(model, compLoader, device)
        –
    else:
        model, args = loadModel(fullPath, device=device)
        model = model.to(device)
        all_logits, trial_lengths, avgDayLoss, cer = original_forward(model, compLoader, device, comp=True)
        
    all_logits = [l.cpu().numpy().astype('float32') for l in all_logits]
    trial_lengths = [l.cpu().numpy().astype('int') for l in trial_lengths]
    all_logits = [l for batch in all_logits for l in list(batch)]
    trial_lengths = [l for batch in trial_lengths for l in list(batch)]
    blank_penalty = np.log(2)
    llm_outputs = run_ngram_model(all_logits, trial_lengths, ngramDecoder, blank_penalty)

    print(len(llm_outputs), len(ground_truth_sentences))

    for i in range(len(llm_outputs)):
        # ground_truth_sentences[i] = ground_truth_sentences[i].strip()
        llm_outputs[i] = llm_outputs[i].strip()

    # for i in range(len(ground_truth_sentences)):
    #     ground_truth_sentences[i] = convert_sentence(ground_truth_sentences[i])

    # cer, wer = utils._cer_and_wer(llm_outputs, ground_truth_sentences, outputType='speech_sil', returnCI=True)

    # print(f"cer: {cer}") # cer average, confidence interval 
    # print(f"wer: {wer}") #wer average, confidence interval

    # wandb.log({'cer': cer, 'wer': wer})

    out_file = os.path.join(fullPath, "comp_outputs_rerun")   # no extension per your spec
    os.makedirs(os.path.dirname(out_file), exist_ok=True)
    with open(out_file, "w", encoding="utf-8") as f:
        f.write("\n".join(llm_outputs))   # one line per LLM output

    # out_file_stats = os.path.join(fullPath, "stats")
    # os.makedirs(os.path.dirname(out_file_stats), exist_ok=True)
    # with open(out_file_stats, "w", encoding="utf-8") as f:
    #     f.write(str(cer) + '\n')
    #     f.write(str(wer) + '\n')


Runnning for neurips_gru_datalog_lr_scheduler_seed_0


/home3/skaasyap/willett/neural_seq_decoder/src/neural_decoder/load_models.py:128: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),


ctc loss: 32.847910, cer: 2.810417
1200 880
Runnning for neurips_gru_datalog_lr_scheduler_seed_1


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


/home3/skaasyap/willett/neural_seq_decoder/src/neural_decoder/load_models.py:128: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),


...

/home3/skaasyap/willett/neural_seq_decoder/src/neural_decoder/load_models.py:128: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),


ctc loss: 33.171315, cer: 2.819688
1200 880
Runnning for neurips_gru_datalog_lr_scheduler_seed_9


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


/home3/skaasyap/willett/neural_seq_decoder/src/neural_decoder/load_models.py:128: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),


ctc loss: 33.221589, cer: 2.815729
1200 880


In [2]:
import pickle
with open('/home3/skaasyap/willett/data_log_both_held_out_days', "rb") as handle:
    loadedData = pickle.load(handle)

In [9]:
t = loadedData['test']
print(t[0]['transcriptions'])

['They do that here.', "She's not feeling too great today.", 'Much more crime than ever before.', 'How does this grab you.', "With Jim's schedule.", 'What do you want to do.', "We didn't even call them.", "Venezuela has the world's largest oil reserves.", 'When we decided to move.', "What's it like from an artist's point of view?", "I don't think they're in a conspiracy.", 'We get upgraded.', 'These are the voyages of the Starship Enterprise.', 'Sort of the ground level.', 'Are you a boy or a girl?', 'I would say about ten or fifteen years ago.', 'This is the first time.', 'I do pull for Chicago.', 'If you have a drug testing program.', 'A lot of people get convicted.', "They're always laying people off.", 'May of last year.', "We've always gone to a lot of movies in our life.", 'Our financial budget.', 'Sort of seems to be taking.', 'Is that putting too fine a point on it?', 'Make or break you.', 'That sort of stuff.', 'But this is usually not the case.', 'We are not real consistent w

In [15]:
trainLoader, testLoader, loadedData = getDatasetLoadersLocal(
    "/home3/skaasyap/willett/data_log_both_held_out_days",
    batch_size,
)

TypeError: 'DataLoader' object is not subscriptable

In [4]:
with open('/home3/skaasyap/willett/data', "rb") as handle:
    loadedData = pickle.load(handle)

modelPath = '/home3/skaasyap/willett/obi_models/outputs/'
modelName = 'neurips_gru_baseline_seed_seed_1'
dataPath = '/home3/skaasyap/willett/data'
model_types = []
model_types.append('r')
device = 'cuda:3'
i=0
print(f"Runnning for {modelName}")

fullPath = f"{modelPath}{modelName}"

args_file = os.path.join(fullPath, "args")
with open(args_file, "rb") as f:
    args = pickle.load(f)

model, args = loadModel(fullPath, device=device)
model = model.to(device)
# all_logits, trial_lengths, avgDayLoss, cer = original_forward(model, compL, device)
all_logits = []

partition = "competition" # "test"
if partition == "competition":
    testDayIdxs = [4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20]
elif partition == "test":
    testDayIdxs = range(len(loadedData[partition]))

for i, testDayIdx in enumerate(testDayIdxs):
    test_ds = SpeechDataset([loadedData[partition][i]])
    test_loader = torch.utils.data.DataLoader(
        test_ds, batch_size=8, shuffle=False, num_workers=0
    )
    for j, (X, y, X_len, y_len, _) in enumerate(test_loader):
        X, y, X_len, y_len, dayIdx = (
            X.to(device),
            y.to(device),
            X_len.to(device),
            y_len.to(device),
            torch.tensor([testDayIdx], dtype=torch.int64).to(device),
        )
        pred = model.forward(X, dayIdx)
        
        all_logits.append(pred)


all_logits = [l.cpu().numpy().astype('float32') for l in all_logits]
trial_lengths = [l.cpu().numpy().astype('int') for l in trial_lengths]
all_logits = [l for batch in all_logits for l in list(batch)]
trial_lengths = [l for batch in trial_lengths for l in list(batch)]
blank_penalty = np.log(2)
llm_outputs = run_ngram_model(all_logits, trial_lengths, ngramDecoder, blank_penalty)

print(len(llm_outputs), len(ground_truth_sentences))

for i in range(len(llm_outputs)):
    # ground_truth_sentences[i] = ground_truth_sentences[i].strip()
    llm_outputs[i] = llm_outputs[i].strip()


out_file = os.path.join(fullPath, "comp_outputs_rerun")   # no extension per your spec
os.makedirs(os.path.dirname(out_file), exist_ok=True)
with open(out_file, "w", encoding="utf-8") as f:
    f.write("\n".join(llm_outputs)) 
        

Runnning for neurips_gru_baseline_seed_seed_1


/home3/skaasyap/miniconda3/envs/willett/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


RuntimeError: stack expects each tensor to be equal size, but got [224, 256] at entry 0 and [454, 256] at entry 1

In [ ]:
function get_output() {
 if [ -z “$1” ]; then
  echo “Usage: get_output <folder_name>”
  return 1
 fi
 # Exclude both optimizer and optimizer_ctc files/directories
 # (Alternatively, use --exclude ‘optimizer*’ to match any name that starts with “optimizer”)
 rsync -av \
    --exclude ‘optimizer’ \
    --exclude ‘optimizer_ctc’ \
    skaasyap@obiwan.ee.ucla.edu:/data/willett_data/outputs/“$1" \
    /Users/shrek/comp_outputs/
}
function send_output() {
 if [ -z “$1” ]; then
  echo “Usage: send_output <folder_name>”
  return 1
 fi
 scp -r /Users/shrek/comp_outputs/$1 skaasyap@leia.ee.ucla.edu:/home3/skaasyap/willett/outputs/
}